In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [2]:
#getting credentials
with open('C:/Users/Sean/.edit/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
#extracting api
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_tasty_wings.json"
JSON_FILE



'Data/results_in_progress_tasty_wings.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
    
    
    ## save an empty list to start the file
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")



[i] Data/results_in_progress_tasty_wings.json not found. Saving empty list to new file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location='Phoenix, AZ',
                                term='Hot Wings',
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results



1500

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [10]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

75

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [12]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [13]:
## Create a new empty json file (exist the previous if it exists)

## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location="Phoenix, AZ",
                                term='Hot Wings',
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



- 20 previous results found.


74

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location='Phoenix, AZ',
                                    term='Hot Wings', 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/74 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,XHnE__Oec_KDfGTJ7jrIpg,valley-wings-phoenix-2,Valley Wings,https://s3-media3.fl.yelpcdn.com/bphoto/hyxklW...,False,https://www.yelp.com/biz/valley-wings-phoenix-...,494,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,"{'latitude': 33.62695, 'longitude': -112.10013}","[pickup, delivery]",$,"{'address1': '15414 N 19th Ave', 'address2': '...",+16025043937,(602) 504-3937,1137.406168
1,MgTEJx28njNh4abS5X50uw,valley-wings-scottsdale,Valley Wings,https://s3-media2.fl.yelpcdn.com/bphoto/ryPZlM...,False,https://www.yelp.com/biz/valley-wings-scottsda...,268,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",5.0,"{'latitude': 33.464427, 'longitude': -111.908399}","[pickup, delivery]",$$,"{'address1': '1495 N Hayden Rd', 'address2': N...",+14806879006,(480) 687-9006,25533.807766
2,HmOsUTXGc3QhRBTyL4Pmig,crazy-mikes-wings-glendale,Crazy Mike's Wings,https://s3-media2.fl.yelpcdn.com/bphoto/OqKjXx...,False,https://www.yelp.com/biz/crazy-mikes-wings-gle...,365,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,"{'latitude': 33.63969499114311, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '5350 W Bell Rd', 'address2': 'St...",+16234636881,(623) 463-6881,7462.735591
3,K1qs4e30yoln6scnm-tmyg,atl-wings-phoenix,ATL Wings,https://s3-media2.fl.yelpcdn.com/bphoto/C2FO36...,False,https://www.yelp.com/biz/atl-wings-phoenix?adj...,339,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 33.47988, 'longitude': -111.99777}",[delivery],$,"{'address1': '3923 E Thomas Rd', 'address2': '...",+16022759464,(602) 275-9464,19348.478838
4,jb-SRwuyska79jPV7FrOfA,epic-wings-glendale,Epic Wings,https://s3-media2.fl.yelpcdn.com/bphoto/skCYM1...,False,https://www.yelp.com/biz/epic-wings-glendale?a...,134,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,"{'latitude': 33.50888209354806, 'longitude': -...",[delivery],$,"{'address1': '9524 W Camelback Rd', 'address2'...",+16027731477,(602) 773-1477,21132.246432


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,x8Sp-h-oR5WdzoG5WEsHnQ,kfc-phoenix-6,KFC,https://s3-media2.fl.yelpcdn.com/bphoto/QJRDy9...,False,https://www.yelp.com/biz/kfc-phoenix-6?adjust_...,79,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"{'latitude': 33.6411582785754, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '4050 E Bell Rd', 'address2': '',...",+16024046498,(602) 404-6498,9278.668082
996,r-1gEc8Xvojo770CaS2gUw,papa-johns-pizza-goodyear-2,Papa Johns Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/GT79Ta...,False,https://www.yelp.com/biz/papa-johns-pizza-good...,22,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.357361325064375, 'longitude': ...",[pickup],$,"{'address1': '17650 W. Elliot Rd.', 'address2'...",+16235727272,(623) 572-7272,43837.881168
997,AkLslZ2ZnURpY37Xtfnu-Q,state-48-funk-house-glendale,State 48 Funk House,https://s3-media1.fl.yelpcdn.com/bphoto/cmg6DM...,False,https://www.yelp.com/biz/state-48-funk-house-g...,308,"[{'alias': 'breweries', 'title': 'Breweries'},...",2.5,"{'latitude': 33.53415324139475, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '6770 N Sunrise Blvd', 'address2'...",+16238774448,(623) 877-4448,19095.691815
998,mygLP5-SCnQwIev6LU9gWg,georges-famous-gyros-scottsdale,George's Famous Gyros,https://s3-media1.fl.yelpcdn.com/bphoto/rtcDYJ...,False,https://www.yelp.com/biz/georges-famous-gyros-...,553,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.5,"{'latitude': 33.45168, 'longitude': -111.91653}",[pickup],$$,"{'address1': '7620 E McKellips Rd', 'address2'...",+14808741354,(480) 874-1354,26105.765941
999,aIvvMGWW-Yy3zPZlnXL6Uw,barros-pizza-glendale-4,Barro's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/OKuxS8...,False,https://www.yelp.com/biz/barros-pizza-glendale...,81,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.6130542030071, 'longitude': -1...",[delivery],$,"{'address1': '13971 N 59th Ave', 'address2': '...",+16024903090,(602) 490-3090,8837.501895


In [18]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_tasty_wings.csv.gz', compression='gzip',index=False)

